# Obtaining information about your `backend`

#### _Note: All the attributes of the backend are described in detail in the [Qiskit Backend Specifications](https://arxiv.org/pdf/1809.03452.pdf). This page reviews a subset of the spec._

Programming a quantum computer at the microwave pulse level requires more information about the device than is required at the circuit level. A quantum circuit is built for an abstract quantum computer -- it will yield the same quantum state on any quantum computer (except for varying performance levels). A pulse schedule, on the other hand, is so specific to the device, that running one program on two different backends is not expected to have the same result, even on perfectly noiseless systems.

As a basic example, imagine a drive pulse `q0_X180` calibrated on qubit 0 to enact an $X180$ pulse, which flips the state of qubit 0. If we use the samples from that pulse on qubit 1 on the same device, or qubit 0 on another device, we do not know what the resulting state will be -- but we can be pretty sure it won't be an $X180$ operation. The qubits are each unique, with various drive coupling strengths. If we have specified a frequency for the drive pulse, it's very probable that pulse would have little effect on another qubit, which has its own resonant frequency.

With that, we have motivated why information from the backend may be very useful at times for building Pulse schedules. The information included in a `backend` is broken into three main parts:

 - **Configuration**: static backend features
 - **Properties**: measured and reported backend characteristics
 - **Defaults**: default settings for the OpenPulse-enabled backend
 
which are each covered in the following sections. While all three of these contain interesting data for Pulse users, the defaults are _only_ provided for backends enabled with OpenPulse.

The first thing you'll need to do is grab a backend to inspect.

In [1]:
from qiskit import IBMQ

provider = IBMQ.load_account()
# We are interested in one that has all the OpenPulse related attributes
backend = provider.backends(open_pulse=True)[0]

## Configuration

The configuration is where you'll find data about the static setup of the device, such as its name, version, the number of qubits, and the types of features it supports.

Let's build a description of our backend using information from the `backend`'s config.

In [2]:
config = backend.configuration()

# Basic Features
print(f"This backend is called {config.backend_name}, and is on version {config.backend_version}. "
      f"It has {config.n_qubits} qubit{'' if config.n_qubits == 1 else 's'}. It "
      f"{'supports' if config.open_pulse else 'does not support'} OpenPulse programs. The basis "
      f"gates supported on this device are {config.basis_gates}.")

This backend is called ibmq_armonk, and is on version 1.1.0. It has 1 qubit. It supports OpenPulse programs. The basis gates supported on this device are ['id', 'u1', 'u2', 'u3'].


Neat! All of the above configuration is available for any backend, whether enabled with OpenPulse or not, although it is not an exhaustive list. There are additional attributes available on Pulse backends. Let's go into a bit more detail with those.

The **timescale**, `dt`, is backend dependent. Think of this as the inverse sampling rate of the control rack's arbitrary waveform generators. Each sample point and duration in a Pulse `Schedule` is given in units of this timescale.

In [3]:
config.dt  # units of seconds

/Users/lauren@ibm.com/code/qiskit-terra/qiskit/providers/models/backendconfiguration.py:355: UserWarning: `dt` and `dtm` now have units of seconds(s) rather than nanoseconds(ns).
  warnings.warn('`dt` and `dtm` now have units of seconds(s) rather '


2.2222222222222221e-10

The configuration also provides information that is useful for building measurements. Pulse supports three measurement levels: `0: RAW`, `1: KERNELED`, and `2: DISCRIMINATED`. The `meas_levels` attribute tells us which of those are supported by this backend. To learn how to execute programs with these different levels, see this page -- COMING SOON.

In [4]:
config.meas_levels

[1, 2]

For backends which support measurement level 0, the sampling rate of the control rack's analog-to-digital converters (ADCs) also becomes relevant. The configuration also has this info, where `dtm` is the time per sample returned:

In [5]:
config.dtm

2.2222222222222221e-10

The measurement map, explained in detail on [this page (IN REVIEW)](adding_measurements.ipynb), is also found here.

In [6]:
config.meas_map

[[0]]

The configuration also supplies convenient methods for getting channels for your schedule programs. For instance:

In [7]:
config.drive(0)

DriveChannel(0)

In [8]:
config.measure(0)

MeasureChannel(0)

In [9]:
config.acquire(0)

AcquireChannel(0)

It is a matter of style and personal preference whether you use `config.drive(0)` or `DriveChannel(0)`.

## Properties

In [10]:
# Conversions from standard SI
us = 1e6
GHz = 1e-9
props = backend.properties()
# Qubit 0 T1 time in microsec
q0_t1_us = props.t1(0) * us
# Qubit 0 T2 time in microsec
q0_t2_us = props.t2(0) * us
# # Gate error for CNOT on qubits 0, 1
# cx_error = props.gate_error('cx', (0, 1))
# Resonant frequency of qubit 0
q2_freq_ghz = props.frequency(0) * GHz
# Duration of the X gate on qubit 0
q1_x_duration_us = props.gate_length('u2', 0) * us

## Defaults

In [11]:
defaults = backend.defaults()

q0_freq = defaults.qubit_freq_est[0]       
q0_meas_freq = defaults.meas_freq_est[0]

inst_map = defaults.instruction_schedule_map

/Users/lauren@ibm.com/code/qiskit-terra/qiskit/providers/models/pulsedefaults.py:155: UserWarning: `qubit_freq_est` and `meas_freq_est` now have units of Hertz(Hz) rather than gigahertz(GHz).
  warnings.warn('`qubit_freq_est` and `meas_freq_est` now have units of '


### Pulse Schedule definitions for QuantumCircuit instructions

In [13]:
inst_map.get('measure', (q for q in range(config.n_qubits)))

inst_map.get('u1', 0, P0=3.1415)

inst_map.has('x', 3)  # Does qubit 3 have an x gate defined on it?

False